In [8]:
%load_ext autotime

import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from heapq import nlargest
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_columns",20)
pd.set_option('float_format', '{:.3f}'.format)

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns


# Load the dataset

In [9]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# pd.options.display.max_columns
pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_columns",20)
pd.set_option('float_format', '{:.3f}'.format)

# Load Usage data
with open('C:\\Users\\natha\Desktop\\Capstone_local_save\\Orginal Dataset\\App_usage_trace.txt') as f:
    lines = f.readlines()#[:100000]
    df = []
    for line in lines:
        data = line.split()
        df.append(data)
usage = pd.DataFrame(df, columns=['uid','timestamp', 'loc', 'app_id','traffic'])
usage['traffic'] = usage['traffic'].astype('float64') / 1e6 # Convert traffic to MB
usage['timestamp'] = usage['timestamp'].apply(lambda x: datetime.strptime(x, "%Y%m%d%H%M%S")) # Convert to datetime object
usage['loc'] = usage['loc'].astype('int64')
usage["date"] = usage["timestamp"].dt.strftime("%m-%d")

time: 1min 6s


In [10]:
# group uid and date, purpose is to see the number of request that a user made for each day
user_usage_perday = usage.groupby(["uid", "date"]).size().reset_index()
# rename columns for easy slicing
user_usage_perday.columns = ["uid", "date", "count"]
# number of users that does not have X counts per day
# change this
required_count = 200

# slice dataset to only remain those rows that has less than 20 count
df_less10day = user_usage_perday[user_usage_perday["count"] < required_count]
# group dataset with uid to get the count of the occurance of the uid, so we can determine which users have just 1 day or more
uid_less10day = df_less10day.groupby(["uid"]).size().reset_index()
# rename columns for easy slicing
uid_less10day.columns = ["uid", "count"]
print("="*100)
print("Current condition: remove user that has less than {} records per day".format(required_count))
print("="*100)
# print("There are {} users that does not meet the requirement". format(uid_less10day.shape[0]))
print("-"*100)
print("There are {} users that have less than {} records for exact 1 day".format((uid_less10day.shape[0]- uid_less10day[uid_less10day["count"] != 1].shape[0]), required_count))
print("-"*100)
print("There are {} users that have less than {} records for more than 1 day".format(uid_less10day[uid_less10day["count"] != 1].shape[0], required_count))
print("-"*100)
print("Under current condition, {} rows will be removed, {:.5%} of the entire dataset ".format(df_less10day["count"].sum(), df_less10day["count"].sum()/4171950))
print("="*100)

Current condition: remove user that has less than 200 records per day
----------------------------------------------------------------------------------------------------
There are 124 users that have less than 200 records for exact 1 day
----------------------------------------------------------------------------------------------------
There are 589 users that have less than 200 records for more than 1 day
----------------------------------------------------------------------------------------------------
Under current condition, 201725 rows will be removed, 4.83527% of the entire dataset 
time: 1.06 s


In [11]:
usage[(usage["uid"] == str(0)) & (usage["date"] == '04-20')]

,uid,timestamp,loc,app_id,traffic,date
0,0,2016-04-20 08:13:19,8194,361,0.002,04-20
1,0,2016-04-20 08:13:20,8194,361,0.002,04-20
2,0,2016-04-20 08:13:22,8194,361,0.004,04-20
3,0,2016-04-20 08:13:30,8194,361,0.001,04-20
4,0,2016-04-20 08:13:31,8194,361,0.002,04-20
5,0,2016-04-20 08:16:44,8194,361,0.002,04-20
6,0,2016-04-20 08:16:49,8194,31,0.002,04-20
7,0,2016-04-20 08:16:49,8194,360,0.002,04-20
8,0,2016-04-20 08:16:49,8194,612,0.002,04-20
9,0,2016-04-20 08:16:50,8194,31,0.003,04-20


time: 469 ms


In [12]:
usage[(usage["uid"] == str(0)) & (usage["date"] == '04-21')]

,uid,timestamp,loc,app_id,traffic,date
87,0,2016-04-21 08:42:21,8194,342,0.030,04-21
88,0,2016-04-21 08:44:12,8194,1,0.008,04-21
89,0,2016-04-21 08:44:47,8194,857,0.027,04-21
90,0,2016-04-21 08:44:48,8194,857,0.002,04-21
91,0,2016-04-21 08:44:49,8194,31,0.009,04-21
92,0,2016-04-21 08:44:49,8194,4,0.002,04-21
93,0,2016-04-21 08:44:49,8194,529,0.002,04-21
94,0,2016-04-21 08:44:49,8194,612,0.001,04-21
95,0,2016-04-21 08:44:49,8194,857,0.579,04-21
96,0,2016-04-21 08:44:50,8194,31,0.307,04-21


time: 453 ms
